In [1]:
from data import preparation
import data.hierarchy as hie
from sklearn.model_selection import train_test_split

In [2]:
data_name = "wiki_small"

In [3]:
with open('data/%s/hierarchy.txt' % data_name, 'w') as f1:
    with open('data/%s/%s.ht' % (data_name,data_name)) as f:
        for l in f:
            split = l.strip().split(':')
            p = split[0]
            c = split[1].split(',')
            for cc in c:
                f1.write('%s %s\n'% (p,cc))

In [4]:
hierarchy, parent_of, all_name, name_to_index, level = hie.reindex_hierarchy(
                '%s/hierarchy.txt' % data_name)
hie.save_hierarchy("%s/hierarchy.pickle" % data_name, hierarchy,
                   parent_of, all_name, name_to_index, level)

In [5]:
# for mode in ['train', 'test']:
#     for i in range(1,6):
#         datas, labels = preparation.import_data('%s/folds/%s_fold%d.dat.%s' % (data_name, data_name, i, mode))
#         new_labels = preparation.map_index_of_label('%s/hierarchy.pickle' % data_name, labels)
#         if mode == 'train':
#             train_data, validate_data, train_target, validate_target = train_test_split(
#                 datas, new_labels, test_size=0.1, random_state=12345)
#             preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, mode), train_data, train_target)
#             preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, "validate"), validate_data, validate_target)
#         else:
#             preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, mode), datas, new_labels)

In [6]:
hierarchy, parent_of, all_name, name_to_index, level_array = hie.reindex_hierarchy(
                '%s/new_hierarchy.txt' % data_name)
hie.save_hierarchy("%s/hierarchy.pickle" % data_name, hierarchy,
                   parent_of, all_name, name_to_index, level_array)

In [7]:
for i in range(len(level_array)-1):
    print(level_array[i+1] - level_array[i])

18
281
1055
1884


In [8]:
level = level_array

# Real Flow 

In [ ]:
from data import Dataset
from embedding import Doc2Vec, NoTag_Doc2Vec, OnlyLeafDoc2Vec
from assemble_classifier import AssembleNoLabel, AssemblePredicted,AssemblePredictedHidden ,AssemblePredictedHiddenAdd
import shutil
import os

%load_ext autoreload
%autoreload 2

In [ ]:
%%time
shl_M = []
shl_m = []
new_M = []
new_m = []
shl_f1 = []
new_f1 = []
for fold in range(4, 6):
    print()
    print()

    dataset_train = Dataset(data_name, fold, "train")
    dataset_validate = Dataset(data_name, fold, "validate")
    dataset_test = Dataset(data_name, fold, "test")
    
    dataset_train.delete_label_more(level[-1])
    dataset_validate.delete_label_more(level[-1])
    dataset_test.delete_label_more(level[-1])
    
    print("train doc2vec")
    doc2vec = Doc2Vec(dataset_train.number_of_classes(), size=150, epoch=180, batch_size=10000)
#     if os.path.isdir('best_now/doc2vec_wiki_small_%d/' % fold):
#         os.makedirs(directory)
#         doc2vec.load_model("best_now/doc2vec_wiki_small_%d/180.model" % fold)
#     else:
    doc2vec.fit(dataset_train.datas, dataset_train.labels, dataset_validate.datas, dataset_validate.labels, early_stopping=False, one_time=True, fold=fold)

    dataset_train.change_to_Doc2Vec(doc2vec)
    dataset_validate.change_to_Doc2Vec(doc2vec)
    dataset_test.change_to_Doc2Vec(doc2vec)
    
    print("train main classifier")
    if os.path.isdir('data/%s/output' % data_name):
        shutil.rmtree('data/%s/output' % data_name)
#     model = AssemblePredicted(data_name, dataset_train, dataset_validate, dataset_test, iteration=2000, stopping_time=200, batch_size=65536, hidden_size=[300,1500,2000,2500], target_hidden_size=[60,100,150], use_dropout=True, start_level=0)
    model = AssemblePredictedHidden(data_name, dataset_train, dataset_validate, dataset_test, iteration=3000, batch_size=600, hidden_size=[300,1500,1500,1500], use_dropout=True, start_level=0, end_level=100)
    model.train(verbose=False)
    model.tuning_threshold()
    print("evaluate")
    f1_macro, f1_micro, f1_each = model.evaluate("test", correction=True)
    shl_M.append(f1_macro)
    shl_m.append(f1_micro)
    shl_f1.append(f1_each)
    print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
    for l, (macro, micro) in enumerate(f1_each):
        print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (l, macro, micro))
        
    print("train main classifier")
    if os.path.isdir('data/%s/output' % data_name):
        shutil.rmtree('data/%s/output' % data_name)
    model = AssemblePredicted(data_name, dataset_train, dataset_validate, dataset_test, iteration=3000, stopping_time=600, batch_size=65536, hidden_size=[336,862,2310,3000], target_hidden_size=[100,100,100], use_dropout=True, start_level=0)
#     model = AssemblePredictedHidden(data_name, dataset_train, dataset_validate, dataset_test, iteration=20000, batch_size=2000, hidden_size=[300,1500,2000,2500], use_dropout=True, start_level=0, end_level=100)
    model.train(verbose=False)
    model.tuning_threshold()
    print("evaluate")
    f1_macro, f1_micro, f1_each = model.evaluate("test", correction=True)
    new_M.append(f1_macro)
    new_m.append(f1_micro)
    new_f1.append(f1_each)
    print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
    for l, (macro, micro) in enumerate(f1_each):
        print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (l, macro, micro))
     



train doc2vec
Doc2Vec by Gensim
0.125
Epoch: 180 Similar: 0.07492
train main classifier
evaluate
F1 macro: 0.2076 F1 micro: 0.2814
Level: 0 F1 macro: 0.2344 F1 micro: 0.2688
Level: 1 F1 macro: 0.2115 F1 micro: 0.2322
Level: 2 F1 macro: 0.2234 F1 micro: 0.2951
Level: 3 F1 macro: 0.1979 F1 micro: 0.3921
train main classifier


In [ ]:
import numpy as np
each_M = np.array([0.] * 4)
each_m = np.array([0.] * 4)
for i in new_f1:
    t = np.array(i).transpose()
    each_M += t[0]
    each_m += t[1]
print(each_M)
print(each_m)

In [ ]:
import numpy as np
each_M = np.array([0.] * 4)
each_m = np.array([0.] * 4)
for i in shl_f1:
    t = np.array(i).transpose()
    each_M += t[0]
    each_m += t[1]
print(each_M)
print(each_m)

In [ ]:
print(sum(shl_m))
print(sum(new_m))

In [ ]:
print(sum(shl_M))
print(sum(new_M))